In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from vector_quantize_pytorch import VectorQuantize

In [17]:
# import torch
# import torch.nn as nn

# class ResidualBlock(nn.Module):
#    def __init__(self, in_dim, out_dim):
#        super().__init__()
#        self.downsample = in_dim != out_dim
#        self.net = nn.Sequential(
#            nn.Linear(in_dim, out_dim),
#            nn.LayerNorm(out_dim),
#            nn.GELU(),
#            nn.Linear(out_dim, out_dim),
#            nn.LayerNorm(out_dim),
#            nn.GELU()
#        )
#        if self.downsample:
#            self.proj = nn.Linear(in_dim, out_dim)
   
#    def forward(self, x):
#        if self.downsample:
#            return self.proj(x) + self.net(x)
#        return x + self.net(x)

# class Encoder(nn.Module):
#    def __init__(self, input_dim=1000, hidden_dims=[512, 384, 256], num_tokens=32, codebook_dim=64):
#        super().__init__()
       
#        # Initial projection with one residual block
#        self.input_proj = ResidualBlock(input_dim, hidden_dims[0])
       
#        # Main network with one residual block per layer
#        layers = []
#        for i in range(len(hidden_dims)-1):
#            layers.append(ResidualBlock(hidden_dims[i], hidden_dims[i+1]))
#        self.layers = nn.Sequential(*layers)
       
#        # Project to token space with one residual block
#        self.token_proj = ResidualBlock(hidden_dims[-1], num_tokens * codebook_dim)
       
#        self.num_tokens = num_tokens
#        self.codebook_dim = codebook_dim
       
#    def forward(self, x):
#        x = self.input_proj(x)
#        x = self.layers(x)
#        x = self.token_proj(x)
#        return x.view(x.shape[0], self.num_tokens, self.codebook_dim)

# class Decoder(nn.Module):
#    def __init__(self, output_dim=1000, hidden_dims=[256, 384, 512], num_tokens=32, codebook_dim=64):
#        super().__init__()
       
#        # Process tokens with one residual block
#        self.token_proj = ResidualBlock(num_tokens * codebook_dim, hidden_dims[0])
       
#        # Main network with one residual block per layer
#        layers = []
#        for i in range(len(hidden_dims)-1):
#            layers.append(ResidualBlock(hidden_dims[i], hidden_dims[i+1]))
#        self.layers = nn.Sequential(*layers)
       
#        # Final projection with one residual block
#        self.output_proj = ResidualBlock(hidden_dims[-1], output_dim)
       
#    def forward(self, x):
#        # x shape: [batch_size, num_tokens, codebook_dim] 
#        x = x.reshape(x.shape[0], -1)  # Flatten tokens
#        x = self.token_proj(x)
#        x = self.layers(x)
#        return self.output_proj(x)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, dim, num_heads=2, dropout=0.1):
        super().__init__()
        assert dim % num_heads == 0, f"dim {dim} must be divisible by num_heads {num_heads}"
        
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5
        
        # Separate projections for Q, K, V
        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        
        self.out_proj = nn.Linear(dim, dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, N, C = x.shape  # batch, num_tokens, channels
        H = self.num_heads

        # Generate Q, K, V with separate projections
        q = self.q_proj(x).reshape(B, N, H, self.head_dim).transpose(1, 2)  # B, H, N, head_dim
        k = self.k_proj(x).reshape(B, N, H, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).reshape(B, N, H, self.head_dim).transpose(1, 2)

        # Attention
        attn = (q @ k.transpose(-2, -1)) * self.scale  # B, H, N, N
        attn = attn.softmax(dim=-1)
        attn = self.dropout(attn)
        
        # Apply attention to V
        out = (attn @ v).transpose(1, 2).reshape(B, N, C)  # B, N, C
        out = self.out_proj(out)
        out = self.dropout(out)
        
        return out

class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads=2, mlp_ratio=2, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.attn = MultiHeadAttention(dim, num_heads, dropout)
        
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden_dim, dim)
        )
        
    def forward(self, x):
        # Pre-norm architecture
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class CustomViTEncoder(nn.Module):
    def __init__(
        self,
        input_dim=1000,
        num_tokens=32,
        token_dim=8,
        num_layers=4,
        num_heads=2,
        dropout=0.1
    ):
        super().__init__()
        self.num_tokens = num_tokens
        self.token_dim = token_dim
        
        # Initial projection and reshape
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, num_tokens * token_dim),
            # nn.LayerNorm(num_tokens * token_dim),
            # nn.GELU(),
            # # nn.Dropout(dropout),
            # nn.Linear(num_tokens * token_dim, num_tokens * token_dim),
            # # nn.Dropout(dropout)
        )
        
        # Positional embedding
        self.pos_embed = nn.Parameter(torch.randn(1, num_tokens, token_dim) * 0.02)  # smaller init
        
        # Transformer layers
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(
                dim=token_dim,
                num_heads=num_heads,
                mlp_ratio=2,
                dropout=dropout
            ) for _ in range(num_layers)
        ])
        
        self.norm = nn.LayerNorm(token_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Project and reshape: [B, 1000] -> [B, 32, 8]
        x = self.input_proj(x)
        x = x.reshape(-1, self.num_tokens, self.token_dim)
        
        # Add positional embeddings and dropout
        x = x + self.pos_embed
        x = self.dropout(x)
        
        # Apply transformer blocks
        # print("input shape: ", x.shape)
        for block in self.transformer_blocks:
            x = block(x)
            
        x = self.norm(x)
        return x

# Mirror image of encoder for decoder
class CustomViTDecoder(nn.Module):
    def __init__(
        self,
        output_dim=1000,
        num_tokens=32,
        token_dim=8,
        num_layers=4,
        num_heads=2,
        dropout=0.1
    ):
        super().__init__()
        self.num_tokens = num_tokens
        self.token_dim = token_dim
        
        # Positional embedding
        self.pos_embed = nn.Parameter(torch.randn(1, num_tokens, token_dim) * 0.02)
        
        # Transformer layers
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(
                dim=token_dim,
                num_heads=num_heads,
                mlp_ratio=2,
                dropout=dropout
            ) for _ in range(num_layers)
        ])
        
        self.norm = nn.LayerNorm(token_dim)
        self.dropout = nn.Dropout(dropout)
        
        # Output projection
        self.output_proj = nn.Sequential(
            nn.Linear(num_tokens * token_dim, output_dim),
            # nn.LayerNorm(num_tokens * token_dim),
            # nn.GELU(),
            # # nn.Dropout(dropout),
            # nn.Linear(num_tokens * token_dim, output_dim)
        )
        
    def forward(self, x):
        # Add positional embeddings and dropout
        x = x + self.pos_embed
        x = self.dropout(x)
        
        # Apply transformer blocks
        for block in self.transformer_blocks:
            x = block(x)
            
        x = self.norm(x)
        
        # Project back to original dimension
        x = x.reshape(x.shape[0], -1)
        x = self.output_proj(x)
        
        return x

In [23]:
import lightning as L
from torchmetrics.functional import pearson_corrcoef

class VQVAE(L.LightningModule):
    def __init__(
            self, 
            input_dim=1000, 
            hidden_dims=[512, 384, 256], 
            num_tokens=32,
            num_layers=2,
            codebook_size=1024, 
            codebook_dim=8,
            commitment_weight=0.25,
            quantizer_decay=0.99,
            learning_rate=3e-4,
            weight_decay=0.01
            ):
        super().__init__()
        
        self.encoder = CustomViTEncoder(num_layers=num_layers, token_dim=codebook_dim)
        self.decoder = CustomViTDecoder(num_layers=num_layers, token_dim=codebook_dim)
        self.quantizer = VectorQuantize(
                dim=codebook_dim,
                codebook_size=codebook_size,
                decay=quantizer_decay,
                commitment_weight=commitment_weight,
                )
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

        #Temperature scheduling for Vector Quantizer
        self.temp = 1.0
        self.min_temp = 0.5
        self.temp_anneal_rate = 0.999

        self.save_hyperparameters()
    
    # def on_train_batch_start(self, batch, batch_idx):
    #     self.temp = max(self.min_temp, self.temp * self.temp_anneal_rate)
        
    def forward(self, x):
        z = self.encoder(x)
        # if self.training:
        #     z = z / self.temp
        z_q, indices, commitment_loss = self.quantizer(z)
        x_recon = self.decoder(z_q)
        return x_recon, commitment_loss, indices
    
    def calculate_metrics(self, x, x_recon):
        # Flatten the tensors for correlation calculation
        x = x.to(torch.float64)
        x_recon = x_recon.to(torch.float64)
        x_flat = x.reshape(x.shape[0], -1)
        x_recon_flat = x_recon.reshape(x_recon.shape[0], -1)
        
        # Calculate Pearson R for each sample in batch
        correlations = torch.stack([
            pearson_corrcoef(x_flat[i], x_recon_flat[i])
            for i in range(x_flat.shape[0])
        ])
        avg_pearson_r = correlations.mean()
        
        # Calculate variance explained
        total_variance = torch.var(x_flat, dim=1).sum()
        residual_variance = torch.var(x_flat - x_recon_flat, dim=1).sum()
        variance_explained = 1 - (residual_variance / total_variance)
        
        return avg_pearson_r, variance_explained


    def training_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        x = batch[0]
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        x_recon, commitment_loss, _ = self(x)
        
        recon_loss = F.mse_loss(x_recon, x)
        total_loss = recon_loss + commitment_loss

        pearson_r, var = self.calculate_metrics(x, x_recon)
        
        # Log metrics
        self.log('train_loss', total_loss)
        self.log('temperature', self.temp)
        self.log('train_recon_loss', recon_loss)
        self.log('train_commitment_loss', commitment_loss)
        self.log('train_pearson_r', pearson_r)
        self.log('train_variance', var)
        
        return total_loss
    
    def validation_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        x = batch[0]
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        x_recon, commitment_loss, _ = self(x)
        
        recon_loss = F.mse_loss(x_recon, x)
        total_loss = recon_loss + commitment_loss

        val_pearson_r, val_var = self.calculate_metrics(x, x_recon)
        
        # Log metrics
        self.log('val_loss', total_loss)
        self.log('val_recon_loss', recon_loss)
        self.log('val_pearson_r', val_pearson_r)
        self.log('val_variance', val_var)
    
    def configure_optimizers(self):
        # Separate parameter groups for different learning rates
        encoder_decoder_params = []
        quantizer_params = []
        
        # Group parameters
        for name, param in self.named_parameters():
            if 'quantizer' in name:
                quantizer_params.append(param)
            else:
                encoder_decoder_params.append(param)
        
        # Create optimizer with parameter groups
        optimizer = torch.optim.AdamW([
            {
                'params': encoder_decoder_params,
                'lr': self.learning_rate,
                'weight_decay': self.weight_decay
            },
            {
                'params': quantizer_params,
                'lr': self.learning_rate * 0.5,  # Lower learning rate for quantizer
                'weight_decay': 0  # Usually no weight decay for quantizer
            }
        ], betas=(0.9, 0.999), eps=1e-8)
        
        # Scheduler setup
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=10,
            min_lr=1e-6,
            verbose=True,
            threshold=1e-4,
            threshold_mode='rel'
        )
        
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",  # Metric to monitor
            }
        }
    


In [20]:
import h5py
import glob
from torch.utils.data import Dataset, DataLoader

class FMRIDataset(Dataset):
    def __init__(self, h5_file):
        data_list = []
        try:
            with h5py.File(h5_file, 'r') as f:
                for key in f.keys():
                    data = torch.from_numpy(f[key][:]).float()
                    data_list.append(data)
        except Exception as e:
            print(f"Error loading {h5_file}: {str(e)}")
        self.data = torch.cat(data_list, dim=0)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Create separate datasets for train and val
train_file = '/home/pranav/mihir/algonauts_challenge/algonauts_2025.competitors/fmri/sub-01/func/sub-01_task-friends_space-MNI152NLin2009cAsym_atlas-Schaefer18_parcel-1000Par7Net_desc-s123456_bold.h5'
val_file = '/home/pranav/mihir/algonauts_challenge/algonauts_2025.competitors/fmri/sub-01/func/sub-01_task-movie10_space-MNI152NLin2009cAsym_atlas-Schaefer18_parcel-1000Par7Net_bold.h5'

train_dataset = FMRIDataset(train_file)
val_dataset = FMRIDataset(val_file)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [21]:
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Training samples: 137913
Validation samples: 24758


In [24]:
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import WandbLogger

run_name = "fmri_1_VIT_vq_08comm_16qdim"
project = "fmri_vq_tokenizer"
wandb_logger = WandbLogger(
    project=project,
    name=run_name,
    save_dir="wandb_logs/"
)
trainer = L.Trainer(
    max_epochs=-1,
    precision=32,
    logger=wandb_logger,
    callbacks=[
        ModelCheckpoint(
            dirpath=f'checkpoints/{project}/{run_name}',
            filename='{epoch:02d}_{val_loss:.3f}',
            monitor='val_loss',
            mode='min',
            save_top_k=1,
            save_last=True
        ),
        EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=10,
            verbose=True
        )
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [25]:
from lightning.pytorch.utilities.model_summary import ModelSummary
torch.set_float32_matmul_precision('high')
model = VQVAE(num_layers=1, codebook_dim=16, codebook_size=1024, commitment_weight=0.8)
summary = ModelSummary(model, max_depth=2)
print(summary)

   | Name                       | Type              | Params | Mode 
--------------------------------------------------------------------------
0  | encoder                    | CustomViTEncoder  | 515 K  | train
1  | encoder.input_proj         | Sequential        | 512 K  | train
2  | encoder.transformer_blocks | ModuleList        | 2.2 K  | train
3  | encoder.norm               | LayerNorm         | 32     | train
4  | encoder.dropout            | Dropout           | 0      | train
5  | decoder                    | CustomViTDecoder  | 515 K  | train
6  | decoder.transformer_blocks | ModuleList        | 2.2 K  | train
7  | decoder.norm               | LayerNorm         | 32     | train
8  | decoder.dropout            | Dropout           | 0      | train
9  | decoder.output_proj        | Sequential        | 513 K  | train
10 | quantizer                  | VectorQuantize    | 0      | train
11 | quantizer.project_in       | Identity          | 0      | train
12 | quantizer.project_out  

In [26]:
trainer.fit(model, train_loader, val_loader)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: mihir-neal (mihirneal). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | encoder   | CustomViTEncoder | 515 K  | train
1 | decoder   | CustomViTDecoder | 515 K  | train
2 | quantizer | VectorQuantize   | 0      | train
-------------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.124     Total estimated model params size (MB)
44        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_pearson_r did not improve in the last 10 records. Best score: 0.800. Signaling Trainer to stop.


In [1]:
import wandb
wandb.finish()


In [27]:
# model = VQVAE()
model = VQVAE.load_from_checkpoint("/home/pranav/mihir/algonauts_challenge/algonauts2025/checkpoints/fmri_vq_tokenizer/fmri_1_VIT_vq_08comm_16qdim/epoch=55_val_pearson_r=0.800.ckpt")
model.eval()
val_data = []
val_recon = []
with torch.no_grad():
    for batch in val_loader:
        x = batch[0].to(model.device)
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        x_recon, _, _ = model(x)
        val_data.append(x)
        val_recon.append(x_recon)

val_data = torch.cat(val_data)
val_recon = torch.cat(val_recon)

#Calculating dead codes and perplexity
from tqdm import tqdm

# Initialize counters
codebook_size = 1024
counts = torch.zeros(codebook_size, device="cuda")  # Use "cpu" if needed

# Run inference on the test set
model.eval()
with torch.no_grad():
    for batch in tqdm(val_loader):
        x = batch.to(model.device)
        z_e = model.encoder(x)  # Get encoder output
        z_q, indices, _ = model.quantizer(z_e)  # Assuming `vq_layer` returns indices
        indices_flat = indices.view(-1)  # Flatten all batch/spatial dimensions
        counts += torch.bincount(indices_flat, minlength=codebook_size)

# Compute metrics
num_dead_codes = (counts == 0).sum().item()
prob = counts / (counts.sum() + 1e-10)
perplexity = torch.exp(-torch.sum(prob * torch.log(prob + 1e-10)))

print(f"Dead codes: {num_dead_codes} / {codebook_size}")
print(f"Perplexity: {perplexity.item():.1f} (max: {codebook_size})")


100%|██████████| 774/774 [00:01<00:00, 536.14it/s]

Dead codes: 112 / 1024
Perplexity: 541.0 (max: 1024)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot code frequencies
plt.figure(figsize=(10, 4))
sns.histplot(counts.cpu().numpy(), bins=50, log_scale=(False, True))
plt.xlabel("Code Frequency (log scale)")
plt.title("Code Usage Distribution")
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

codebook_vectors = model.quantizer.codebook.cpu().numpy()  # Shape [1024, 8]
similarity = cosine_similarity(codebook_vectors)

plt.figure(figsize=(8, 8))
plt.imshow(similarity, cmap="viridis", vmin=-1, vmax=1)
plt.colorbar(label="Cosine Similarity")
plt.title("Codebook Vector Similarity")
plt.show()

In [ ]:
from sklearn.manifold import TSNE

# Collect codebook vectors and their usage counts
codebook = model.quantizer.codebook.cpu().numpy()
counts_np = counts.cpu().numpy()

# Run t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
codebook_2d = tsne.fit_transform(codebook)

# Plot with color indicating usage frequency
plt.figure(figsize=(10, 8))
plt.scatter(codebook_2d[:, 0], codebook_2d[:, 1], c=np.log(counts_np + 1), cmap="viridis")
plt.colorbar(label="Log(Usage Count + 1)")
plt.title("Codebook t-SNE Projection")
plt.show()

In [28]:
var_explained = 1 - torch.var(val_data - val_recon) / torch.var(val_data)
pearson_r = torch.corrcoef(torch.stack([val_data.flatten(), val_recon.flatten()]))[0,1]
mse = torch.mean((val_data - val_recon) ** 2)
mae = torch.mean(torch.abs(val_data - val_recon))

print(f"Variance explained: {var_explained:.3f}")
print(f"Pearson correlation: {pearson_r:.3f}")
print(f"MSE: {mse:.3f}")
print(f"MAE: {mae:.3f}")

Variance explained: 0.663
Pearson correlation: 0.818
MSE: 0.124
MAE: 0.278


In [29]:
def analyze_codebook(model, val_loader, device='cuda'):
    model.eval()
    usage_count = torch.zeros(model.quantizer.codebook_size, device=device)
    total_tokens = 0
    distances = []
    
    with torch.no_grad():
        for batch in val_loader:
            x = batch[0].to(device)
            if len(x.shape) == 1:
                x = x.unsqueeze(0)
            # Get encoder output
            z = model.encoder(x)
            # Get quantizer outputs before quantization
            z_flat = z.reshape(-1, z.shape[-1])
            _, indices, _ = model.quantizer(z_flat)
            
            # Update usage count
            unique, counts = torch.unique(indices, return_counts=True)
            usage_count[unique] += counts
            total_tokens += indices.numel()
            
            # Calculate distances to assigned codebook vectors
            dists = torch.cdist(z_flat, model.quantizer.codebook)
            distances.append(dists.min(dim=1)[0])
    
    # Calculate metrics
    usage_prob = usage_count / total_tokens
    active_codes = (usage_count > 0).sum().item()
    entropy = -(usage_prob * torch.log2(usage_prob + 1e-10)).sum().item()
    distances = torch.cat(distances)
    
    print(f"Active codebook vectors: {active_codes}/{model.quantizer.codebook_size}")
    print(f"Codebook entropy: {entropy:.2f} bits")
    print(f"Mean distance to codebook: {distances.mean():.4f}")
    print(f"Distance std: {distances.std():.4f}")
    
    return {
        'usage_count': usage_count.cpu(),
        'usage_prob': usage_prob.cpu(),
        'active_codes': active_codes,
        'entropy': entropy,
        'distances': distances.cpu(),
    }

In [30]:
analyze_codebook(model, val_loader)

Active codebook vectors: 819/1024
Codebook entropy: 9.07 bits
Mean distance to codebook: 0.0616
Distance std: 0.0351


{'usage_count': tensor([  0.,   4.,   1.,  ...,  42., 104.,  10.]),
 'usage_prob': tensor([0.0000e+00, 1.6150e-04, 4.0375e-05,  ..., 1.6957e-03, 4.1990e-03,
         4.0375e-04]),
 'active_codes': 819,
 'entropy': 9.067659378051758,
 'distances': tensor([0.0697, 0.0960, 0.0509,  ..., 0.0380, 0.0438, 0.0454])}